In [10]:
import cv2 as cv
import numpy as np
import pyautogui as pg

#switch off top-left corner failsafe
pg.FAILSAFE = False
pg.PAUSE = 0

In [13]:
cam = cv.VideoCapture(0)
#Blue color limits - for cursor control
lower_blue = np.array([90,100,160])
higher_blue = np.array([125,255,255])
#Green color limts - for clicking
lower_green = np.array([60,100,80])
higher_green = np.array([90,255,255])

ret=True
prev = ''

while(ret):
    #read camera images
    ret,frame = cam.read()
    #flip about y axis
    frame = cv.flip(frame,1)
    
    w = frame.shape[1]
    h = frame.shape[0]
    
    img_smooth = cv.GaussianBlur(frame,(7,7),0)
    #creating rectange in the frame:
    cv.rectangle(frame,(100,100),(400,400),(0,0,255),3)
    cv.rectangle(frame,(200,100),(300,400),(0,0,255),2)
    cv.rectangle(frame,(100,200),(400,300),(0,0,255),2)
    #define ROI 
    #same shape as frame
    mask = np.zeros_like(frame)
    #create white region to extract that from frame
    mask[100:400,100:400] = (255,255,255)
    img_roi = cv.bitwise_and(img_smooth, mask)
    img_hsv = cv.cvtColor(img_roi,cv.COLOR_BGR2HSV)
    #extracting pixels within specified range
    img_threshold = cv.inRange(img_hsv,lower_blue,higher_blue)
    contours,hierarchy = cv.findContours(img_threshold,cv.RETR_TREE,cv.CHAIN_APPROX_NONE)
    #find contour with largest area (if contours present)
    if(len(contours)!=0):
        areas = [cv.contourArea(c) for c in contours]
        max_index = np.argmax(areas)
        cnt = contours[max_index]
        
        M = cv.moments(cnt)
        if(M['m00']!=0):
            #calculating centroid
            c_x = int(M['m10']/M['m00'])
            c_y = int(M['m01']/M['m00'])
            
            #finding location
            #along y-axis
            if(c_y<=200):
                move_y= -5
                curr = 'Upper '
            elif(200<c_y<=300):
                move_y= 0
                curr = 'Middle '
            elif(c_y>300):
                move_y= 5
                curr = 'Bottom '
            #along x-axis
            if(c_x<=200):
                move_x= -5
                curr = curr + 'Left'
            elif(200<c_x<=300):
                move_x= 0
                curr = curr + 'Center'
            elif(c_x>300):
                move_x= 5
                curr = curr + 'Right'
            #move the cursor
            pg.moveRel(move_x,move_y,duration = 0)
            #formatting the output so that it looks nice
            if(prev!=curr):
                print(curr)
                prev = curr
            
            # creating pointer on our object
            center = (c_x,c_y)
            #negative thickness means a filled circle
            cv.circle(frame,center,4,(0,255,0),-1)
    else:
        if(len(prev)>0):
            print('Pointer not in frame')
        prev = ''
    #checking for green colored object for left click
    img_thresh_green = cv.inRange(img_hsv,lower_green,higher_green)
    contours,hierarchy = cv.findContours(img_thresh_green,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)
    if len(contours)!=0:
        pg.leftClick()
        pg.sleep(1)
        print('Clicked')
    cv.imshow('Video',frame)
    #cv.imshow('Thresh',img_threshold)
    if(cv.waitKey(10)==27):
        break
        
cam.release()
cv.destroyAllWindows()

Bottom Right
Bottom Center
Middle Center
Bottom Center
Middle Center
Upper Center
Middle Center
Upper Center
Middle Center
Middle Right
Middle Center
Middle Left
Bottom Left
Middle Left
Bottom Left
Middle Left
Bottom Left
Pointer not in frame
Bottom Left
Pointer not in frame
Bottom Left
Pointer not in frame
Bottom Left
Middle Left
Upper Left
Pointer not in frame
Upper Left
Middle Left
Middle Center
Middle Right
Clicked
Clicked
Clicked
Clicked
Bottom Center
Clicked
Pointer not in frame
Bottom Right
Pointer not in frame
Bottom Center
Middle Center
Upper Center
Upper Right
Middle Right
Middle Center
Bottom Center
Bottom Left
Bottom Center
Middle Center
Middle Right
Bottom Right
Middle Right
Middle Center
Upper Center
Upper Left
Middle Left
Middle Center
Middle Left
Upper Left
Upper Center
Middle Center
Pointer not in frame
Middle Right
Middle Center
Middle Right
Upper Right
Upper Center
Middle Center
Upper Center
Pointer not in frame
Middle Center
Bottom Center
Middle Center
Bottom Center